In [ ]:
import re

import graphviz
import pydot
import pygraphviz as pgv
from IPython.display import display


def is_valid_dot(dot_string):
    """Check if the dot string is a valid graph."""
    try:
        pgv.AGraph(string=dot_string)
        return True  # If no exception is raised, then dot_string is a valid graph representation
    except:
        return False


def all_nodes_are_rectangles(dot_string):
    """Check if all nodes in the graph are rectangles."""
    graphs = pydot.graph_from_dot_data(dot_string)
    if not graphs:
        return False
    graph = graphs[0]
    for node in graph.get_nodes():
        if node.get_shape() != "record":
            return False
    return True


def verify_label_syntax(dot_string):
    try:
        graph = pgv.AGraph(string=dot_string)
    except Exception as e:
        return f"Invalid DOT string: {e}"

    # Regular expression to match the label syntax
    label_pattern = re.compile(r"^\{\{[<>\w\s|]+\} \| [\w: ]+ \| \{[<>\w\s|]+\}\}$")

    for node_name in graph.nodes():
        node = graph.get_node(node_name)
        label = node.attr["label"]
        if label:
            label = label.strip('"')
            if not label_pattern.match(label):
                return f"Invalid label syntax for node {node_name}: {label}"

    return True


def verify_ports(dot_string):
    """Parses a DOT string to extract port information from node labels,
    checks if the port lists are sorted in ascending order and are unique.

    If all labels are valid, returns True.
    If not, returns a list of invalid labels.
    """
    graph = pgv.AGraph(string=dot_string)

    invalid_labels = []
    label_pattern = re.compile(r"\{\{([^\}]+)\}\s*\|\s*([^\|]+)\|\s*\{([^\}]+)\}\}")

    for node in graph.nodes():
        label = node.attr["label"]
        if label:
            match = label_pattern.match(label.strip('"'))
            if match:
                left_ports = [
                    int(port[1:]) for port in re.findall(r"<(\w+)>", match.group(1))
                ]
                right_ports = [
                    int(port[1:]) for port in re.findall(r"<(\w+)>", match.group(3))
                ]
                concatenated_ports = list(reversed(left_ports)) + right_ports
                is_sorted_and_unique = concatenated_ports == sorted(
                    concatenated_ports
                ) and len(concatenated_ports) == len(set(concatenated_ports))

                if not is_sorted_and_unique:
                    invalid_labels.append(label.strip('"'))
            else:
                return f"Invalid label structure for node {node.name}: {label}"

    return True if not invalid_labels else invalid_labels


# Example usage
dot_data = """
graph cascaded_mzi_circuit {
  rankdir=LR;
  node [shape=record];

  C1 [label="{{<o2> o2|<o1> o1} | C1: dc_mzi_2x2_heater_doped_si_cband | {<o3> o3|<o4> o4}}"];
  C2 [label="{{<o2> o2|<o1> o1} | C2: dc_mzi_2x2_heater_doped_si_cband | {<o3> o3|<o4> o4}}"];

  C1:o3 -- C2:o1;
  C1:o4 -- C2:o2;
}
"""

print("is_valid_dot:", is_valid_dot(dot_data))
print("all_nodes_are_rectangles:", all_nodes_are_rectangles(dot_data))
print("validate_label_syntax:", verify_label_syntax(dot_data))

print("verify_ports:", verify_ports(dot_data))

dot = graphviz.Source(dot_data)
display(dot)
# dot.render(view=True)

In [ ]:
import graphviz
from IPython.display import display


def check_crossings(dot_string):
    """Check if a Graphviz dot string has any crossing edges.

    This function takes a dot string as input, applies a layout algorithm to position
    the nodes and edges, and then checks for any crossing edges in the graph.
    """
    # Load the dot string
    graph = pgv.AGraph(string=dot_string)

    # Apply a layout to the graph
    graph.layout(prog="dot")

    # Get edge coordinates
    edges = []
    for edge in graph.edges():
        points = edge.attr["pos"].split()
        start = tuple(map(float, points[0].split(",")))
        end = tuple(map(float, points[-1].split(",")))
        edges.append((start, end))

    # Function to check if two line segments (p1, q1) and (p2, q2) intersect
    def do_intersect(p1, q1, p2, q2):
        def orientation(p, q, r):
            val = (q[1] - p[1]) * (r[0] - q[0]) - (q[0] - p[0]) * (r[1] - q[1])
            if val == 0:
                return 0
            return 1 if val > 0 else 2

        def on_segment(p, q, r):
            if min(p[0], r[0]) <= q[0] <= max(p[0], r[0]) and min(p[1], r[1]) <= q[
                1
            ] <= max(p[1], r[1]):
                return True
            return False

        o1 = orientation(p1, q1, p2)
        o2 = orientation(p1, q1, q2)
        o3 = orientation(p2, q2, p1)
        o4 = orientation(p2, q2, q1)

        if o1 != o2 and o3 != o4:
            return True

        if o1 == 0 and on_segment(p1, p2, q1):
            return True
        if o2 == 0 and on_segment(p1, q2, q1):
            return True
        if o3 == 0 and on_segment(p2, p1, q2):
            return True
        if o4 == 0 and on_segment(p2, q1, q2):
            return True

        return False

    # Check each pair of edges for intersection
    crossings = []
    for i, (p1, q1) in enumerate(edges):
        for j, (p2, q2) in enumerate(edges):
            if i != j and do_intersect(p1, q1, p2, q2):
                crossings.append(((p1, q1), (p2, q2)))

    return crossings
    # return True if crossings else False


dot_string = """
graph cascaded_mzis {
  rankdir=LR;
  node [shape=record];

  C1 [label="{{<o2> o2|<o1> o1} | C1: dc_mzi_2x2_heater_doped_si_cband | {<o3> o3|<o4> o4}}"];
  C2 [label="{{<o2> o2|<o1> o1} | C2: dc_mzi_2x2_heater_doped_si_cband | {<o3> o3|<o4> o4}}"];

  C1:o3 -- C2:o1;
  C1:o4 -- C2:o2;
}
"""

dot_string = """
graph new_circuit {
  rankdir=LR;
  node [shape=record];

  C1 [label="{{<o2> o2|<o1> o1} | C1: dc_mzi_2x2_heater_doped_si_cband | {<o3> o3|<o4> o4}}"];
  C2 [label="{{<o2> o2|<o1> o1} | C2: _directional_coupler | {<o3> o3|<o4> o4}}"];

  C1:o3 -- C2:o1;
  C1:o4 -- C2:o2;
}
"""

dot = graphviz.Source(dot_string)
display(dot)

print(check_crossings(dot_string))

In [ ]:
import yaml


def edges_dot_to_yaml(dot_graph, yaml_netlist):
    # Regular expression to find the edges
    edge_pattern = re.compile(r"(\w+):(\w+) -- (\w+):(\w+);")

    # Find all matches in the DOT graph string
    edges = edge_pattern.findall(dot_graph)

    # Format edges as required
    formatted_edges = [f"{edge[0]},{edge[1]}: {edge[2]},{edge[3]}" for edge in edges]

    netlist = yaml.safe_load(yaml_netlist)
    if "reasoning" in netlist:
        del netlist["reasoning"]
    netlist["routes"] = {}
    netlist["routes"]["optical"] = {}
    netlist["routes"]["optical"]["links"] = {}

    for edge in formatted_edges:
        source, target = edge.split(": ")
        netlist["routes"]["optical"]["links"][source] = target

    # Dump the updated netlist back to a YAML string
    updated_netlist = yaml.dump(netlist, sort_keys=False)

    return updated_netlist


dot_string = """
graph new_circuit {
  rankdir=LR;
  node [shape=record];

  C1 [label="{{<o2> o2|<o1> o1} | C1: dc_mzi_2x2_heater_doped_si_cband | {<o3> o3|<o4> o4}}"];
  C2 [label="{{<o2> o2|<o1> o1} | C2: _directional_coupler | {<o3> o3|<o4> o4}}"];

  C1:o3 -- C2:o2;
  C1:o4 -- C2:o1;
}
"""

netlist = """instances:
  C1:
    component: dc_mzi_2x2_heater_doped_si_cband
    info:
      ports: 2x2
  C2:
    component: _directional_coupler
    info:
      ports: 2x2
name: new_circuit
placements:
  C1:
    x: 0
    y: 0
  C2:
    x: 374.0
    y: 198.4
routes:
  optical:
    links:
      C1,o3: C2,o1
      C1,o4: C2,o2"""

print(edges_dot_to_yaml(dot_string, netlist))